# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [ ]:
# replace the code below with your pulse rate algorithm.
import glob

import numpy as np
import scipy as sp
import scipy.io

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl 

import scipy.signal

# sampling rate
fs = 125

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    # only return the ppg, x, y, z 
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(signal, pass_band, fs):
    """Bandpass Filter.
    # based on Udacity class notes
    
    Args:
        signal: (np.array) The input signal
        pass_band: (tuple) The pass band. Frequency components outside 
            the two elements in the tuple will be removed.
        fs: (number) The sampling rate of <signal>
        
    Returns:
        (np.array) The filtered signal
    """
    # NEW
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Estimates pulse rate and Confidence  
    
    Args: 
        date_fl: (str) filepath to troika *.mat file 
    Returns: 
        errors: Calculated errors for each heart rate 
        Confidence: Calculated Confidence for each heart rate 
    """
    # Load data using LoadTroikaDataFile
    # photoplethysmogram (PPG) 
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    #print(ppg)
    
    # Compute pulse rate estimates and estimation confidence.
    # NEW
    
    # All signals were sampled at 125 Hz.
    fs = 125 
    
    # win_len 
    # total time 12 sec; 8 win_len will give a sliding windows 
    # [0-8), [2-10), [4-12) based on discussion forum discussion
    win_len = 8 
    
    # win_shift
    win_shift = 2
    nfft_window = fs * 8 
    noverlap = fs * 6 
    
    # filter the signal
    filtered_ppg = BandpassFilter(ppg, (40/60.0,240/60.0), fs=fs)   

    accx = BandpassFilter(accx, (40/60.0,240/60.0), fs=fs)
    accy = BandpassFilter(accy, (40/60.0,240/60.0), fs=fs) 
    accz = BandpassFilter(accz, (40/60.0,240/60.0), fs=fs) 
    
    # acc magnitude based on accelerometer deep-dive lecture 
    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    
    # extract the ground truth as a flattened vector 
    true_vals = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1)
    
    # call predict_heart_rate to get predictions and confidence 
    predictions, confidence = predict_heart_rate(acc, ppg, fs)

    errors = np.abs(predictions - true_vals)
    
    # estimate errors and confidence and return them
    
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    # errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
    
    return np.array(errors), np.array(confidence)


def FT(acc_s, fs):
    """
    Take a Fourier Transform (FT) of a signal and return the frequencies
    Args: 
        accx: (numpy array) signal 
        fs: (int) Sampling frequency in Hertz 
    Returns: 
        fft: (float) FT magnitudes
        freqs: (numpy array) frequencies 
    """
    # NEW
    
    # based on activity_classifier_utils.py code and class notes
    # fft_len = max(len(acc_s), 2046)
    # zero padding gives
    fft_len = len(acc_s)
    
    # create an array of frequency bins 
    fft_freqs = np.fft.rfftfreq(fft_len, 1/fs)
    
    fft = np.fft.rfft(acc_s, fft_len)
    fft = np.abs(fft)
    
    return fft_freqs, fft
    
def predict_heart_rate(acc, ppg, fs,
                       window_len = 8, 
                       window_shift = 2, 
                       min_bp = 40/60.0, 
                       max_bp = 240/60.0 ):
    
#     estimates pulse rate from the PPG signal and a 3-axis accelerometer. 
#     assumes pulse rate will be restricted between 40 BPM (min_bp; beats per minute) and 240 BPM
#     produces an estimation confidence. A higher confidence value means that this 
#     (max_bp) estimate should be more accurate than an estimate with a lower confidence value.
#     produces an output at least every 2 seconds.

#   

    # create empty list to add later 
    confidence = []
    bpm0_pred = []
    
    window_len = window_len * fs 
    window_shift = window_shift * fs 
    # MAE was higher
    window_hw = 1
    # window_hw = 40.0/60.0 # didnt work
    
    # loop over the samples
    
    wfrom = len(ppg) - window_len + 1 
    wto = window_shift
    
    for i in range(0, wfrom, wto ): 
        
        # Let us now focus on the current window 
        ppg_w = ppg[i:i+window_len]
        acc_w = acc[i:i+window_len]
        
        #Let us do FT 
        ppg_fs, ppg_fft = FT(ppg_w, fs)
        acc_fs, acc_fft = FT(acc_w, fs)
        
        # restrict to only pulse rate indicated by the paper 
        # Remember to bandpass filter all your signals. 
        # Use the 40-240BPM range to create your pass band.

        acc_fft[acc_fs <= min_bp] = 0.0
        acc_fft[acc_fs >= max_bp] = 0.0
        
        ppg_fft[ppg_fs <= min_bp] = 0.0
        ppg_fft[ppg_fs >= max_bp] = 0.0
        
        
        # You can plot your estimates on top of the spectrogram to 
        # see where things are going wrong.
        
#         plt.figure(figsize=(12,8))
#         plt.specgram(acc_fft, Fs=fs, NFFT=250, noverlap=125, 
#                     xextent=[0, len(acc_fft)/fs/60])
#         plt.show()
        
        # When the dominant accelerometer frequency is 
        # the same as the PPG, try picking the next strongest PPG 
        # frequency if there is another good candidate.
        
        ppg_freq = ppg_fs[np.argmax(ppg_fft, axis=0)]
        acc_freq = acc_fs[np.argmax(acc_fft, axis=0)]
        
        ffreq = ppg_freq
        # Confidence calculator 
        # take ppg_fft, ppg_freq and ppg_fs
        
        cond1 = (ppg_fs >= ppg_freq - window_hw) 
        cond2 = (ppg_fs <= ppg_freq + window_hw)

        w = cond1 & cond2
        abs_ppg_fft = np.abs(ppg_fft)
        conf = np.sum(abs_ppg_fft[w])/np.sum(abs_ppg_fft)
        # check to see whether the freq are the same 
        
        if abs(ppg_freq - acc_freq) == 0:
            # pick the next strongest one 
            temp_freq = ppg_fs[np.argsort(ppg_fft,axis=0)[-2]]
            # cal new conf
            cond1 = (ppg_fs >= temp_freq - window_hw) 
            cond2 = (ppg_fs <= temp_freq + window_hw)
            w = cond1 & cond2
            abs_ppg_fft = np.abs(ppg_fft)
            temp_conf = np.sum(abs_ppg_fft[w])/np.sum(abs_ppg_fft)
            
            if (temp_conf > conf):
                ffreq = temp_freq
                conf = temp_conf
            
        bpm0_pred.append(ffreq * 60) 
        confidence.append(conf)
    
    return bpm0_pred, confidence
